# Sensitivity of Fermentation Metrics to Start Conditions #

### File to generate Fig S5 and Fig S6 ###

In [1]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.core.Fermentation import TwoStageFermentation, OneStageFermentation
import mcpecaso as mcpecaso
import os
from mcpecaso.core import mcPECASO

mcpecaso.settings.time_end = 100
mcpecaso.settings.initial_biomass = 0.05
mcpecaso.settings.initial_substrate = 500
mcpecaso.settings.num_timepoints = 10000

num_cores = multiprocessing.cpu_count()

In [2]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


In [3]:
wt_model = cameo.models.bigg.iJO1366
model = cameo.models.bigg.iJO1366

Academic license - for non-commercial use only


In [4]:
def fermentation_conditions_variant_mcpecaso(pecaso_template, initial_biomass, initial_substrate):
    temp = deepcopy(pecaso_template)
    pecaso_dict = {}
    temp.settings.initial_substrate = initial_substrate
    temp.settings.initial_biomass = initial_biomass
    temp.calculate_fermentation_characteristics()
    pecaso_dict_biomass['initial_biomass'] = initial_biomass
    pecaso_dict_biomass['initial_substrate'] = initial_biomass
    if temp.two_stage_best_batch:
        pecaso_dict['ts_best'] = {}
        pecaso_dict['ts_best']['batch_productivity'] = temp.two_stage_best_batch.batch_productivity
        pecaso_dict['ts_best']['batch_yield'] = temp.two_stage_best_batch.batch_yield
        pecaso_dict['ts_best']['batch_titer'] = temp.two_stage_best_batch.batch_titer

    else:
        pecaso_dict['ts_best'] = None

    if temp.one_stage_best_batch:
        pecaso_dict['os_best'] = {}
        pecaso_dict['os_best']['batch_productivity'] = temp.one_stage_best_batch.batch_productivity
        pecaso_dict['os_best']['batch_yield'] = temp.one_stage_best_batch.batch_yield
        pecaso_dict['os_best']['batch_titer'] = temp.one_stage_best_batch.batch_titer

    else:
        pecaso_dict['os_best'] = None
    
    if temp.two_stage_suboptimal_batch:
        pecaso_dict['ts_sub'] = {}
        pecaso_dict['ts_sub']['batch_productivity'] = temp.two_stage_suboptimal_batch.batch_productivity
        pecaso_dict['ts_sub']['batch_yield'] = temp.two_stage_suboptimal_batch.batch_yield
        pecaso_dict['ts_sub']['batch_titer'] = temp.two_stage_suboptimal_batch.batch_titer

    else:
        pecaso_dict['ts_sub'] = None

    return pecaso_dict

In [5]:
substrate_concentrations = [50, 100, 250, 500, 1000]
pecaso_template = mcPECASO(model=model,biomass_rxn=model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                           substrate_rxn=model.reactions.EX_glc__D_e, target_rxn=model.reactions.EX_lac__D_e,
                           condition='Target - D-Lactate')
biomass_range = np.logspace(-6,2,64)


The model is complete.


c:\users\kraj5\onedrive - university of toronto\research\phd_uoft\python\tsdyssco\tsdyssco\core\substrate_dependent_envelopes.py:72: UserWarning:

The parameters used with the model for substrate uptake resulted in rates that are lower than thee minimum feasible uptake for one or more cases. The minimum feasible uptake rate was used in these cases



In [6]:
pickling_off = open("biomass_sensitivity.pickle","rb")
pecaso_dict = pickle.load(pickling_off)
pickling_off.close()

In [7]:
xaxis_titles = ['Productivity<br>(mmol/L.h)']
for j,metric in enumerate(['productivity']):
    titles = ["Initial Substrate: <br>"+str(sub_conc)+" (mM)" for sub_conc in pecaso_dict.keys()]
    fig = subplots.make_subplots(rows=1, cols=5, subplot_titles=titles,print_grid=False, horizontal_spacing=0.05)
    for i,sub_conc in enumerate(pecaso_dict.keys()):

        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'][:-15], 
                                 y=pecaso_dict[sub_conc]['ts_best'][metric][:-15],
                                 name='Optimized Two Stage',
                                 legendgroup='ts_best', line={'color': '#8DC447'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'][:-15], 
                                 y=pecaso_dict[sub_conc]['ts_sub'][metric][:-15],
                                 name='Traditional Two Stage',
                                 legendgroup='ts_sub', line={'color': '#FF5555'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'][:-15], 
                                 y=pecaso_dict[sub_conc]['os_best'][metric][:-15],
                                 name='Best One Stage',
                                 legendgroup='os_best', line={'color': '#76D1F4'},
                                 showlegend=False), row=1, col=i+1)
        fig.update_yaxes(range=(0,1.1*max(max(pecaso_dict[sub_conc]['ts_best'][metric][:-15]),
                                          max(pecaso_dict[sub_conc]['os_best'][metric][:-15]),
                                          max(pecaso_dict[sub_conc]['ts_sub'][metric][:-15]))),row=1, col=i+1)
        if i==0:
            fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                             title_text=xaxis_titles[j],
                             title_font=dict(size=12, color='black'),
                             title_standoff=0,
                             automargin=True, row=1, col=i+1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_text='Initial Biomass<br>Concentration (g/L)',
                     title_font=dict(size=12, color='black'),
                     tickfont=dict(size=10, color='black'),
                     title_standoff=0, showexponent='all',exponentformat='power',
                     range=(-6,0),
                     automargin=False,
                     ticks='outside',
                     type='log',
                     nticks=5,
                     showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     tickangle=0,
                     ticks='outside',
                     tickfont=dict(size=12, color='black'),
                     showgrid=False)
    fig.update_layout(width=950,
                      height=325,
                      margin={'l':8})

    plot(fig)
if not os.path.exists('images'):
    os.mkdir('images')
#pio.write_image(fig, 'images/biomass_sensitivity_objA.svg')   

In [8]:
pickling_off = open("substrate_sensitivity.pickle","rb")
pecaso_dict = pickle.load(pickling_off)
pickling_off.close()

In [9]:
xaxis_titles = ['Productivity<br>(mmol/L.h)']
for j,metric in enumerate(['productivity']):
    titles = ["Initial Biomass: <br>"+str(sub_conc)+" (mM)" for sub_conc in pecaso_dict.keys()]
    fig = subplots.make_subplots(rows=1, cols=5, subplot_titles=titles,print_grid=False, horizontal_spacing=0.05)
    for i,sub_conc in enumerate(pecaso_dict.keys()):

        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'], 
                                 y=pecaso_dict[sub_conc]['ts_best'][metric],
                                 name='Optimized Two Stage',
                                 legendgroup='ts_best', line={'color': '#8DC447'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'], 
                                 y=pecaso_dict[sub_conc]['ts_sub'][metric],
                                 name='Traditional Two Stage',
                                 legendgroup='ts_sub', line={'color': '#FF5555'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'], 
                                 y=pecaso_dict[sub_conc]['os_best'][metric],
                                 name='Best One Stage',
                                 legendgroup='os_best', line={'color': '#76D1F4'},
                                 showlegend=False), row=1, col=i+1)
        fig.update_yaxes(range=(0,1.1*max(max(pecaso_dict[sub_conc]['ts_best'][metric]),
                                          max(pecaso_dict[sub_conc]['os_best'][metric]),
                                          max(pecaso_dict[sub_conc]['ts_sub'][metric]))),row=1, col=i+1)
        if i==0:
            fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                             title_text=xaxis_titles[j],
                             title_font=dict(size=12, color='black'),
                             title_standoff=0,
                             automargin=True, row=1, col=i+1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_text='Initial Substrate<br>Concentration (mmol/L)',
                     title_font=dict(size=12, color='black'),
                     tickfont=dict(size=10, color='black'),
                     title_standoff=0, showexponent='all',exponentformat='power',
                     range=(0,1000),
                     automargin=False,
                     ticks='outside',
                     nticks=5,
                     showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     tickangle=0,
                     ticks='outside',
                     tickfont=dict(size=12, color='black'),
                     showgrid=False)
    fig.update_layout(width=950,
                      height=325,
                      margin={'l':8})
    #fig.update_annotations(dict(font=dict(size=16, color='black')))

    plot(fig)
if not os.path.exists('images'):
    os.mkdir('images')
#pio.write_image(fig, 'images/substrate_sensitivity_objA.svg')   

In [10]:
pickling_off = open("biomass_sensitivity_yield_constrained_prod.pickle","rb")
pecaso_dict = pickle.load(pickling_off)
pickling_off.close()

In [11]:
xaxis_titles = ['Productivity<br>(mmol/L.h)']
for j,metric in enumerate(['productivity']):
    titles = ["Initial Substrate: <br>"+str(sub_conc)+" (mM)" for sub_conc in pecaso_dict.keys()]
    fig = subplots.make_subplots(rows=1, cols=5, subplot_titles=titles,print_grid=False, horizontal_spacing=0.05)
    for i,sub_conc in enumerate(pecaso_dict.keys()):

        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'], 
                                 y=pecaso_dict[sub_conc]['ts_best'][metric],
                                 name='Optimized Two Stage',
                                 legendgroup='ts_best', line={'color': '#8DC447'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'], 
                                 y=pecaso_dict[sub_conc]['ts_sub'][metric],
                                 name='Traditional Two Stage',
                                 legendgroup='ts_sub', line={'color': '#FF5555'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'], 
                                 y=pecaso_dict[sub_conc]['os_best'][metric],
                                 name='Best One Stage',
                                 legendgroup='os_best', line={'color': '#76D1F4'},
                                 showlegend=False), row=1, col=i+1)
        fig.update_yaxes(range=(0,1.1*max(max(pecaso_dict[sub_conc]['ts_best'][metric]),
                                          max(pecaso_dict[sub_conc]['os_best'][metric]),
                                          max(pecaso_dict[sub_conc]['ts_sub'][metric]))),row=1, col=i+1)
        if i==0:
            fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                             title_text=xaxis_titles[j],
                             title_font=dict(size=12, color='black'),
                             title_standoff=0,
                             automargin=True, row=1, col=i+1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_text='Initial Biomass<br>Concentration (g/L)',
                     title_font=dict(size=12, color='black'),
                     tickfont=dict(size=10, color='black'),
                     title_standoff=0, showexponent='all',exponentformat='power',
                     range=(-6,0),
                     automargin=False,
                     ticks='outside',
                     type='log',
                     nticks=5,
                     showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     tickangle=0,
                     ticks='outside',
                     tickfont=dict(size=12, color='black'),
                     showgrid=False)
    fig.update_layout(width=950,
                      height=325,
                      margin={'l':8})
    #fig.update_annotations(dict(font=dict(size=16, color='black')))

    plot(fig)
if not os.path.exists('images'):
    os.mkdir('images')
#pio.write_image(fig, 'images/biomass_sensitivity_objB.svg')   

In [12]:
pickling_off = open("substrate_sensitivity_yield_constrained_prod.pickle","rb")
pecaso_dict = pickle.load(pickling_off)
pickling_off.close()

In [13]:
xaxis_titles = ['Productivity<br>(mmol/L.h)']
for j,metric in enumerate(['productivity']):
    titles = ["Initial Biomass: <br>"+str(sub_conc)+" (mM)" for sub_conc in pecaso_dict.keys()]
    fig = subplots.make_subplots(rows=1, cols=5, subplot_titles=titles,print_grid=False, horizontal_spacing=0.05)
    for i,sub_conc in enumerate(pecaso_dict.keys()):

        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'], 
                                 y=pecaso_dict[sub_conc]['ts_best'][metric],
                                 name='Optimized Two Stage',
                                 legendgroup='ts_best', line={'color': '#8DC447'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'], 
                                 y=pecaso_dict[sub_conc]['ts_sub'][metric],
                                 name='Traditional Two Stage',
                                 legendgroup='ts_sub', line={'color': '#FF5555'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'], 
                                 y=pecaso_dict[sub_conc]['os_best'][metric],
                                 name='Best One Stage',
                                 legendgroup='os_best', line={'color': '#76D1F4'},
                                 showlegend=False), row=1, col=i+1)
        fig.update_yaxes(range=(0,1.1*max(max(pecaso_dict[sub_conc]['ts_best'][metric]),
                                          max(pecaso_dict[sub_conc]['os_best'][metric]),
                                          max(pecaso_dict[sub_conc]['ts_sub'][metric]))),row=1, col=i+1)
        if i==0:
            fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                             title_text=xaxis_titles[j],
                             title_font=dict(size=12, color='black'),
                             title_standoff=0,
                             automargin=True, row=1, col=i+1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_text='Initial Substrate<br>Concentration (mmol/L)',
                     title_font=dict(size=12, color='black'),
                     tickfont=dict(size=10, color='black'),
                     title_standoff=0, showexponent='all',exponentformat='power',
                     range=(0,1000),
                     automargin=False,
                     ticks='outside',
                     nticks=5,
                     showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     tickangle=0,
                     ticks='outside',
                     tickfont=dict(size=12, color='black'),
                     showgrid=False)
    fig.update_layout(width=950,
                      height=325,
                      margin={'l':8})
    #fig.update_annotations(dict(font=dict(size=16, color='black')))

    plot(fig)
if not os.path.exists('images'):
    os.mkdir('images')
#pio.write_image(fig, 'images/substrate_sensitivity_objB.svg')   

In [28]:
pickling_off = open("biomass_sensitivity_prod_constrained_yield.pickle","rb")
pecaso_dict = pickle.load(pickling_off)
pickling_off.close()

In [30]:
xaxis_titles = ['Yield<br>(mmol product/mmol substrate)']
for j,metric in enumerate(['yield']):
    titles = ["Initial Substrate: <br>"+str(sub_conc)+" (mM)" for sub_conc in pecaso_dict.keys()]
    fig = subplots.make_subplots(rows=1, cols=5, subplot_titles=titles,print_grid=False, horizontal_spacing=0.05)
    for i,sub_conc in enumerate(pecaso_dict.keys()):

        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'][::], 
                                 y=pecaso_dict[sub_conc]['ts_best'][metric][::],
                                 name='Optimized Two Stage',
                                 legendgroup='ts_best', line={'color': '#8DC447'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'][::4], 
                                 y=pecaso_dict[sub_conc]['ts_sub'][metric][::4],
                                 name='Traditional Two Stage',
                                 legendgroup='ts_sub', line={'color': '#FF5555'}, mode='lines',
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_biomass'][::], 
                                 y=pecaso_dict[sub_conc]['os_best'][metric][::],
                                 name='Best One Stage',
                                 legendgroup='os_best', line={'color': '#76D1F4'},
                                 showlegend=False), row=1, col=i+1)
        fig.update_yaxes(range=(0,1.1*max(max(pecaso_dict[sub_conc]['ts_best'][metric]),
                                          max(pecaso_dict[sub_conc]['os_best'][metric]))),row=1, col=i+1)
        if i==0:
            fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                             title_text=xaxis_titles[j],
                             title_font=dict(size=12, color='black'),
                             title_standoff=0,
                             automargin=True, row=1, col=i+1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_text='Initial Biomass<br>Concentration (g/L)',
                     title_font=dict(size=12, color='black'),
                     tickfont=dict(size=10, color='black'),
                     title_standoff=0, showexponent='all',exponentformat='power',
                     range=(-6,0),
                     automargin=False,
                     ticks='outside',
                     type='log',
                     nticks=5,
                     showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     tickangle=0,
                     ticks='outside',
                     tickfont=dict(size=12, color='black'),
                     showgrid=False)
    fig.update_layout(width=950,
                      height=325,
                      margin={'l':8})
    #fig.update_annotations(dict(font=dict(size=16, color='black')))

    plot(fig)
if not os.path.exists('images'):
    os.mkdir('images')
#pio.write_image(fig, 'images/biomass_sensitivity_objC.svg')   

In [32]:
pickling_off = open("substrate_sensitivity_prod_constrained_yield.pickle","rb")
pecaso_dict = pickle.load(pickling_off)
pickling_off.close()

In [39]:
xaxis_titles = ['Yield<br>(mmol product/mmol substrate)']
for j,metric in enumerate(['yield']):
    titles = ["Initial Biomass: <br>"+str(sub_conc)+" (mM)" for sub_conc in pecaso_dict.keys()]
    fig = subplots.make_subplots(rows=1, cols=5, subplot_titles=titles,print_grid=False, horizontal_spacing=0.05)
    for i,sub_conc in enumerate(pecaso_dict.keys()):

        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'][::], 
                                 y=pecaso_dict[sub_conc]['ts_best'][metric][::],
                                 name='Optimized Two Stage',
                                 legendgroup='ts_best', line={'color': '#8DC447'},
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'][::8], 
                                 y=pecaso_dict[sub_conc]['ts_sub'][metric][::8],
                                 name='Traditional Two Stage',
                                 legendgroup='ts_sub', line={'color': '#FF5555'}, mode='lines',
                                 showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=pecaso_dict[sub_conc]['initial_substrate'][::], 
                                 y=pecaso_dict[sub_conc]['os_best'][metric][::],
                                 name='Best One Stage',
                                 legendgroup='os_best', line={'color': '#76D1F4'},
                                 showlegend=False), row=1, col=i+1)
        fig.update_yaxes(range=(0,1.1*max(max(pecaso_dict[sub_conc]['ts_best'][metric]),
                                          max(pecaso_dict[sub_conc]['os_best'][metric]),
                                          max(pecaso_dict[sub_conc]['ts_sub'][metric]))),row=1, col=i+1)
        if i==0:
            fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                             title_text=xaxis_titles[j],
                             title_font=dict(size=12, color='black'),
                             title_standoff=0,
                             automargin=True, row=1, col=i+1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_text='Initial Substrate<br>Concentration (mmol/L)',
                     title_font=dict(size=12, color='black'),
                     tickfont=dict(size=10, color='black'),
                     title_standoff=0, showexponent='all',exponentformat='power',
                     range=(0,1000),
                     automargin=False,
                     ticks='outside',
                     nticks=5,
                     showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     tickangle=0,
                     ticks='outside',
                     tickfont=dict(size=12, color='black'),
                     showgrid=False)
    fig.update_layout(width=950,
                      height=325,
                      margin={'l':8})
    #fig.update_annotations(dict(font=dict(size=16, color='black')))

    plot(fig)
if not os.path.exists('images'):
    os.mkdir('images')
#pio.write_image(fig, 'images/substrate_sensitivity_objC.svg')   